In [1]:
import discogs_client

In [2]:
def readUserToken():
    try:
        with open("user_token.env", "r") as file:
            content = file.read()
            return content
    except FileNotFoundError:
        return {}
        
d = discogs_client.Client('dardok_app/0.1', user_token='{}'.format(readUserToken().rstrip()))

In [3]:
artist_id = 5531548
min_average = 4.8

In [4]:
print(dir(d))

result_artist = d.artist(artist_id)

print()
print(dir(result_artist))

print()
print(dir(result_artist.releases))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_access_token_url', '_authorize_url', '_base_url', '_check_user_agent', '_delete', '_fetcher', '_get', '_patch', '_post', '_put', '_request', '_request_token_url', 'artist', 'backoff_enabled', 'connection_timeout', 'fee_for', 'get_access_token', 'get_authorize_url', 'identity', 'label', 'list', 'listing', 'master', 'order', 'read_timeout', 'release', 'search', 'set_consumer_key', 'set_timeout', 'set_token', 'user', 'user_agent', 'verbose']

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__

In [5]:
# COMMON FUNCTIONS

def getFilteredReleaseListFromArtist(artist):
    """ A function that returns all the filtered releases from artist """
    filtered_release_list = []
    print()
    print("Processing artist: {} {}...".format(artist.name, artist.url))
    for k in range(1, result_artist.releases.pages+1):
        print()
        print("Page: {}/{}".format(k,result_artist.releases.pages))
        print(result_artist.releases.page(k))
        for item in result_artist.releases.page(k):
            release = d.release(item.id)       
            if release.community.rating.average >= min_average and release.formats[0]['name']=='Vinyl' and 'Electronic' in release.genres:
                filtered_release_list.append(release)
    print(filtered_release_list)
    # return filtered_release_list


In [6]:
getFilteredReleaseListFromArtist(result_artist)


Processing artist: Verraco https://www.discogs.com/artist/5531548-Verraco...

Page: 1/1
[<Release 9556777 'Resistir'>, <Master 1961887 'New Army of Androgynes'>, <Release 11488886 "Don't Kill'em All">, <Release 18238102 'RA.671'>, <Master 1831655 'Grial'>, <Release 20855698 'Grial Remixes'>, <Release 28342054 'Escándaloo'>, <Master 1852017 'Megapunk (Remixes)'>, <Release 25257778 'Herrensauna Vol. 2'>, <Master 2781065 'Pliegues'>, <Master 2306245 'Tresor 30'>, <Release 12136071 'Medellín Rave Society'>, <Release 12542203 'Curated by DSH #099'>, <Release 14908504 'Música Urgente'>, <Release 14908647 'Despertar: Chile'>, <Release 18952612 'Horizontes - Vol. 4'>, <Master 2306245 'Tresor 30'>, <Release 23906087 'Desire Path'>, <Release 24936322 'no pare, sigue sigue'>, <Release 27287130 'Atlantic Stereo'>, <Release 27437412 'Desire Path'>, <Release 29356837 'Fact Mix 850'>, <Master 2872324 'Fabric Presents Nicola Cruz'>, <Master 3079121 'Fabric Selects I'>, <Release 29068300 'Los Totes 2'

HTTPError: 404: Release not found.